# 第10章: 機械翻訳
本章では，日本語と英語の翻訳コーパスである[京都フリー翻訳タスク (KFTT)](http://www.phontron.com/kftt/index-ja.html)を用い，ニューラル機械翻訳モデルを構築する．ニューラル機械翻訳モデルの構築には，[fairseq](https://github.com/pytorch/fairseq)，[Hugging Face Transformers](https://github.com/huggingface/transformers)，[OpenNMT-py](https://github.com/OpenNMT/OpenNMT-py)などの既存のツールを活用せよ．

## 90. データの準備
機械翻訳のデータセットをダウンロードせよ．訓練データ，開発データ，評価データを整形し，必要に応じてトークン化などの前処理を行うこと．ただし，この段階ではトークンの単位として形態素（日本語）および単語（英語）を採用せよ．

GPUの使用制限がかかるので，データの永続化をする  
（ドライブに作業フォルダを保存する）

In [2]:
# 作業用フォルダの作成
from google.colab import drive 
drive.mount('/content/drive')
!mkdir -p '/content/drive/My Drive/work/'
%cd '/content/drive/My Drive/work/'

Mounted at /content/drive
/content/drive/My Drive/work


In [ ]:
## データセットダウンロード
!wget http://www.phontron.com/kftt/download/kftt-data-1.0.tar.gz

--2021-07-19 12:26:33--  http://www.phontron.com/kftt/download/kftt-data-1.0.tar.gz
Resolving www.phontron.com (www.phontron.com)... 208.113.196.149
Connecting to www.phontron.com (www.phontron.com)|208.113.196.149|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 99246893 (95M) [application/gzip]
Saving to: ‘kftt-data-1.0.tar.gz’

kftt-data-1.0.tar.g 100%[===================>]  94.65M  67.3MB/s    in 1.4s    

2021-07-19 12:26:34 (67.3 MB/s) - ‘kftt-data-1.0.tar.gz’ saved [99246893/99246893]



In [3]:
!gunzip kftt-data-1.0.tar.gz

gzip: kftt-data-1.0.tar.gz: No such file or directory


In [4]:
!tar -xvf kftt-data-1.0.tar

tar: kftt-data-1.0.tar: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [ ]:
!head -10 ./kftt-data-1.0/data/tok/kyoto-train.en

Known as Sesshu ( 1420 - 1506 ) , he was an ink painter and Zen monk active in the Muromachi period in the latter half of the 15th century , and was called a master painter .
He revolutionized the Japanese ink painting .
He was given the posthumous name " Toyo " or " Sesshu ( 拙宗 ) . "
Born in Bicchu Province , he moved to Suo Province after entering SShokoku-ji Temple in Kyoto .
Later he accompanied a mission to Ming Dynasty China and learned Chinese ink painting .
His works were many , including not only Chinese-style landscape paintings , but also portraits and pictures of flowers and birds .
His bold compositions and strong brush strokes constituted an extremely distinctive style .
6 of his extant works are designated national treasures . Indeed , he is considered to be extraordinary among Japanese painters .
For this reason , there are a great many artworks that are attributed to him , such as folding screens with pictures of flowers and that birds are painted on them .
There are m

In [ ]:
!head -10 ./kftt-data-1.0/data/tok/kyoto-train.ja

雪舟 （ せっしゅう 、 1420 年 （ 応永 27 年 ） - 1506 年 （ 永正 3 年 ） ） は 号 で 、 15 世紀 後半 室町 時代 に 活躍 し た 水墨 画 家 ・ 禅僧 で 、 画聖 と も 称え られ る 。
日本 の 水墨 画 を 一変 さ せ た 。
諱 は 「 等楊 （ とうよう ） 」 、 もしくは 「 拙宗 （ せっしゅう ） 」 と 号 し た 。
備中 国 に 生まれ 、 京都 ・ 相国 寺 に 入 っ て から 周防 国 に 移 る 。
その 後 遣明 使 に 随行 し て 中国 （ 明 ） に 渡 っ て 中国 の 水墨 画 を 学 ん だ 。
作品 は 数 多 く 、 中国 風 の 山水 画 だけ で な く 人物 画 や 花鳥 画 も よ く し た 。
大胆 な 構図 と 力強 い 筆線 は 非常 に 個性 的 な 画風 を 作り出 し て い る 。
現存 する 作品 の うち 6 点 が 国宝 に 指定 さ れ て お り 、 日本 の 画家 の なか で も 別格 の 評価 を 受け て い る と いえ る 。
この ため 、 花鳥 図 屏風 など に 「 伝 雪舟 筆 」 さ れ る 作品 は 大変 多 い 。
真筆 で あ る か 専門 家 の 間 で も 意見 の 分かれ る もの も 多々 あ る 。


### 前処理

### どうやら，[fairseq](https://github.com/pytorch/fairseq)が便利らしい．
コマンドを打つだけで，学習や予測ができるので

### fairseqとは
Facebookの人工知能研究チームが開発している，機械翻訳用のフレームワークです．Facebookが開発元ということもあり，Pytorchがベースになっています．  
最近はHuggingfaceのTransformersが人気でTransformerモデルを扱うならPytorchを使用していたため，こちらをフレームワークとして選びました．  
その他の機械翻訳フレームワークとしては，MarianNMT，OpenNMT（こちらもPytorchベース）などがあります．基本的な機能はどのフレームワークも大差ない印象ですが，論文実装のコードはfairseqが選ばれている場合が多いらしいです．

In [5]:
## fairseqのインストール
!pip install fairseq

     |████████████████████████████████| 1.7 MB 4.9 MB/s 
     |████████████████████████████████| 144 kB 19.5 MB/s 
     |████████████████████████████████| 54 kB 3.2 MB/s 
     |████████████████████████████████| 112 kB 19.5 MB/s 
     |████████████████████████████████| 74 kB 2.8 MB/s 
     |████████████████████████████████| 636 kB 19.7 MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141229 sha256=3a3a0bf9fe8c749bfde5d61f355aa5f62eb0481be5ad9e051302118fa6c44773
  Stored in directory: /root/.cache/pip/wheels/ca/33/b7/336836125fc9bb4ceaa4376d8abca10ca8bc84ddc824baea6c
Successfully built antlr4-python3-runtime
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [6]:
import os

In [ ]:
!mkdir ./data-bin
!mkdir ./data-bin/kftt.ja-en

fairseq-preprocessはテキストデータをバイナリ化して保存します。今回はすでにspaceでtokenizeされているのでspaceを使いました。

In [ ]:
!fairseq-preprocess -s ja -t en \
    --trainpref  /content/kftt-data-1.0/data/tok/kyoto-train \
    --validpref  /content/kftt-data-1.0/data/tok/kyoto-dev \
    --testpref /content/kftt-data-1.0/data/tok/kyoto-test \
    --destdir data91  \
    --thresholdsrc 5 \
    --thresholdtgt 5 \
    --workers 20

2021-07-19 12:33:26 | INFO | fairseq_cli.preprocess | Namespace(align_suffix=None, alignfile=None, all_gather_list_size=16384, bf16=False, bpe=None, checkpoint_shard_count=1, checkpoint_suffix='', cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='data91', empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=False, optimizer=None, padding_factor=8, profile=False, quantization_config_path=None, scoring='bleu', seed=1, source_lang='ja', srcdict=None, target_lang='en', task='translation', tensorboard_logdir=None, testpref='/content/kftt-data-1.0/data/tok/kyoto-test', tgtdict=None, threshold_loss_scale=None, thresholdsrc=5, thresholdtgt=5, tokenizer=N

In [ ]:
!ls kftt-data-1.0/data/tok/

kyoto-dev.en  kyoto-test.en  kyoto-train.cln.en  kyoto-train.en  kyoto-tune.en
kyoto-dev.ja  kyoto-test.ja  kyoto-train.cln.ja  kyoto-train.ja  kyoto-tune.ja


## 91. 機械翻訳モデルの訓練
90で準備したデータを用いて，ニューラル機械翻訳のモデルを学習せよ（ニューラルネットワークのモデルはTransformerやLSTMなど適当に選んでよい）．

学習の設定は以下に箇条書きした．  
（ハイパーパラメータは後にチューニングする）

*   モデル：LSTM
*   optimizer : adam
*   学習率：1e-5
*   ドロップアウト：0.3
*   最大エポック数：5
*   損失：交差エントロピー











In [ ]:
os.system("mkdir checkpoints")
os.system("mkdir checkpoints/kftt.ja-en")

256

In [ ]:
!pip install sacremoses

In [ ]:
!fairseq-train data91 \
    --save-dir checkpoints/kftt.ja-en/ \
    --arch lstm --share-decoder-input-output-embed \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
    --lr 5e-4 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
    --dropout 0.3 --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-tokens 4096 \
    --max-epoch 5 \
    --eval-bleu \
    --eval-bleu-detok moses \
    --eval-bleu-remove-bpe \
    --eval-bleu-print-samples \
    --best-checkpoint-metric bleu --maximize-best-checkpoint-metric

## --eval-bleu-detok moses \ トークン化解除
##　--eval-bleu-remove-bpe \　BLEUを計算する前にBPEを削除する
##　--eval-bleu-print-samples \　検証中にサンプルした世代を出力する
##　--best-checkpoint-metric bleu \　最良のチェックポイントを保存するために使用するメトリック=bleuに指定
##　--maximize-best-checkpoint-metric 

2021-07-19 07:33:18 | INFO | fairseq_cli.train | Namespace(adam_betas='(0.9, 0.98)', adam_eps=1e-08, adaptive_softmax_cutoff='10000,50000,200000', all_gather_list_size=16384, arch='lstm', batch_size=None, batch_size_valid=None, best_checkpoint_metric='bleu', bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', clip_norm=0.0, cpu=False, criterion='label_smoothed_cross_entropy', curriculum=0, data='data91', data_buffer_size=10, dataset_impl=None, ddp_backend='c10d', decoder_attention='1', decoder_dropout_in=0.3, decoder_dropout_out=0.3, decoder_embed_dim=512, decoder_embed_path=None, decoder_freeze_embed=False, decoder_hidden_size=512, decoder_layers=1, decoder_out_embed_dim=512, device_id=0, disable_validation=False, distributed_backend='nccl', distributed_init_method=None, distributed_no_spawn=False, distributed_num_procs=1, distributed_port=-1, distributed_rank=0, distributed_world_size=1, distributed_wrapper='DDP', dropout=0



*   ./data-bin/: 第一引数はモデルの保存先を指定します。これがデフォルトのようです

*   --lr: learning rateはデータ、バッチサイズなどで大きく適切な値が変わります。1e-5がよく使われているらしいです．

*   --warmup-updates: このパラメータの回数分のminibatchを処理した段階でlrが最大値になります。
1epochのminibatchの数は後述のパラメータで大きく変わるので、変更し忘れないように注意







In [ ]:
## epoch=10で再度やってみる
!fairseq-train data91 \
    --save-dir checkpoints/kftt.ja-en/epoch10/ \
    --arch lstm --share-decoder-input-output-embed \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
    --lr 5e-4 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
    --dropout 0.3 --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-tokens 4096 \
    --max-epoch 10 \
    --eval-bleu \
    --eval-bleu-detok moses \
    --eval-bleu-remove-bpe \
    --eval-bleu-print-samples \
    --best-checkpoint-metric bleu --maximize-best-checkpoint-metric

2021-07-19 12:37:45 | INFO | fairseq_cli.train | Namespace(adam_betas='(0.9, 0.98)', adam_eps=1e-08, adaptive_softmax_cutoff='10000,50000,200000', all_gather_list_size=16384, arch='lstm', batch_size=None, batch_size_valid=None, best_checkpoint_metric='bleu', bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', clip_norm=0.0, cpu=False, criterion='label_smoothed_cross_entropy', curriculum=0, data='data91', data_buffer_size=10, dataset_impl=None, ddp_backend='c10d', decoder_attention='1', decoder_dropout_in=0.3, decoder_dropout_out=0.3, decoder_embed_dim=512, decoder_embed_path=None, decoder_freeze_embed=False, decoder_hidden_size=512, decoder_layers=1, decoder_out_embed_dim=512, device_id=0, disable_validation=False, distributed_backend='nccl', distributed_init_method=None, distributed_no_spawn=False, distributed_num_procs=0, distributed_port=-1, distributed_rank=0, distributed_world_size=1, distributed_wrapper='DDP', dropout=0

## 92. 機械翻訳モデルの適用
91で学習したニューラル機械翻訳モデルを用い，与えられた（任意の）日本語の文を英語に翻訳するプログラムを実装せよ．



fairseq-interactiveを使って，翻訳する．  

https://fairseq.readthedocs.io/en/latest/command_line_tools.html#fairseq-interactive

In [ ]:
!fairseq-interactive data91/ \
  --path checkpoints/kftt.ja-en/checkpoint_best.pt \
  < /content/kftt-data-1.0/data/tok/kyoto-test.ja \
  > 92.out

256

## 93. BLEUスコアの計測
91で学習したニューラル機械翻訳モデルの品質を調べるため，評価データにおけるBLEUスコアを測定せよ．



In [ ]:
!fairseq-score --sys 92.out --ref data-bin/kftt-data-1.0/data/tok/kyoto-test.en

## 94. ビーム探索
91で学習したニューラル機械翻訳モデルで翻訳文をデコードする際に，ビーム探索を導入せよ．ビーム幅を1から100くらいまで適当に変化させながら，開発セット上のBLEUスコアの変化をプロットせよ．



## 95. サブワード化
トークンの単位を単語や形態素からサブワードに変更し，91-94の実験を再度実施せよ．



## 96. 学習過程の可視化
Tensorboardなどのツールを用い，ニューラル機械翻訳モデルが学習されていく過程を可視化せよ．可視化する項目としては，学習データにおける損失関数の値とBLEUスコア，開発データにおける損失関数の値とBLEUスコアなどを採用せよ．



## 97. ハイパー・パラメータの調整
ニューラルネットワークのモデルや，そのハイパーパラメータを変更しつつ，開発データにおけるBLEUスコアが最大となるモデルとハイパーパラメータを求めよ．